#### 라이브러리

In [ ]:
!pip install --upgrade openai
!pip install tiktoken
!pip install openai pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.2 MB/s eta 0:00:00


In [ ]:
import os
import openai
from openai import OpenAI
import tiktoken
import numpy as np
import pandas as pd
import sys
import re

#### chatgpt api

In [ ]:
# 발급받은 API 키 설정
OPENAI_API_KEY = ##API key
client=OpenAI(api_key=OPENAI_API_KEY)

#### prompt

In [ ]:
# file import
df = pd.read_csv('/content/drive/MyDrive/DS541/park_info_evaluation.csv') # 파일 경로

with open('/content/drive/MyDrive/DS541/parkinfo.txt', 'r', encoding='utf-8') as file:
    park_info = file.read()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def evaluate_answer(question, model_answer, baseline_answer):
    prompt = f"""
    You are a judge evaluating two chatbot answers based on the following criteria:
    1. Accuracy: How correct and factually accurate is the response?
    2. Completeness: Does the response answer the full question and provide enough detail?
    3. Readability: Is the language clear, fluent, and easy to understand?
    4. Relevance: Does the response stay on topic and address the question directly?

    Please evaluate the following answers:
    Question: {question}
    Model Answer: {model_answer}
    Baseline Answer: {baseline_answer}

    For each of the four criteria, rate both answers on a scale of 1 to 10, where 1 is the lowest and 10 is the highest.
    Return a score for both answers and your reasoning for each category.
    Please print out the form in the following format: r"(\d+)\. \*\*(.*?) \(Score: (\d+)\)\n"
    For example, Accuracy (Score: 9)

    In addition, here is the actual information that might help you evaluate the answers, especially accuracy:
    {park_info}.
    Please read it very carefully before evaluating the answers.
    """
    chat = client.chat.completions.create(
      model = "gpt-4o-mini-2024-07-18",
      messages = [
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': prompt}],
      max_tokens=500,
      temperature=0.5
    )

    return print(chat.choices[0].message.content)

### evaluation - park info

In [ ]:
output_file = open("evaluation_results.txt", "w")
sys.stdout = output_file

In [ ]:
for index, row in df.iterrows():
    question = row['Question']
    model_answer = row['Model Answer']
    baseline_answer = row['Baseline Answer']

    print(f"#### Question Number {index+1} #### \n")
    evaluation = evaluate_answer(question, model_answer, baseline_answer)
    print('\n\n\n\n\n')

print("Evaluation complete")

In [ ]:
# 평가 결과 텍스트 파일로 저쟝

sys.stdout = sys.__stdout__
output_file.close()

print("Results saved to evaluation_results.txt")